In [ ]:
%load_ext autoreload
%autoreload 0

In [ ]:
%autoreload

In [ ]:
import sys
sys.path.append('/pollard/home/bsmith/Projects/haplo-benchmark/include/StrainFacts')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from lib.pandas_util import idxwhere
import sfacts as sf
import numpy as np
import xarray as xr
import seaborn as sns
from scipy.spatial.distance import cdist
import lib.plot

In [ ]:
# data/ucfmt.sp-104345.metagenotype.filt-poly05-cvrg05.fit-sfacts44_v-s30-g10000-seed0.world.nc

species_id = '104345'

mgen_stem = f'data/ucfmt.sp-{species_id}.metagenotype.filt-poly05-cvrg05'
scg_path = f'data/ucfmt.sp-{species_id}.derep.genotype.nc'

# fit_path = (f'data/ucfmt.sp-{species_id}.metagenotype'
#             f'.filt-poly05-cvrg05'
#             f'.fit-sfacts47_c1e5-s30-g5000-seed1'
# #             f'.refit-sfacts41-g10000-seed0'
#             f'.world.nc'
#            )

fit_path = f'{mgen_stem}.fit-sfacts44_v-s30-g5000-seed0.refit-sfacts41-g10000-seed0.world.nc'
# fit2_path = (f'data/ucfmt.sp-{species_id}.metagenotype'
#             f'.filt-poly05-cvrg05'
#             f'.fit-sfacts47_c1e5-s30-g5000-seed1'
# #             f'.refit-sfacts41-g10000-seed0'
#             f'.world.nc'
#            )

scg_to_sample_path = f'data/ucfmt.sp-{species_id}.derep.barcode_to_sample.tsv'
library_to_sample_path = "data/ucfmt.barcode_to_sample.tsv"
rabund_threshold = 0.01
pseudo = 0
horizontal_coverage_threshold = 0.05

mgen = sf.data.Metagenotypes.load(f'{mgen_stem}.nc')
drplt = sf.data.Metagenotypes.load(scg_path)
inference = sf.data.World.load(fit_path)




# inference2 = sf.data.World.load(fit2_path)


In [ ]:
inference.sizes

In [ ]:
mgen_consensus = mgen.to_estimated_genotypes(pseudo=pseudo, fillna=False)
scg = drplt.to_estimated_genotypes(pseudo=pseudo, fillna=False)


inferred_geno = inference.genotypes
inferred_comm = inference.communities

# inferred_geno2 = inference2.genotypes
# inferred_comm2 = inference2.communities

position = list(set(scg.position.values) & set(inferred_geno.position.values))
mgen_consensus, scg, inferred_geno = [
    x.mlift("sel", position=position) for x in [mgen_consensus, scg, inferred_geno]
]

mgen_entropy = mgen.entropy("sample")
scg_entropy = drplt.entropy("sample")
comm_entropy = inferred_comm.entropy("sample")
geno_entropy = inferred_geno.entropy("strain")
scg_horizontal_coverage = drplt.horizontal_coverage()
mgen_horizontal_coverage = mgen.horizontal_coverage()

library_to_sample = pd.read_table(
    library_to_sample_path, index_col="barcode", squeeze=True
)
scg_to_sample = pd.read_table(
    scg_to_sample_path, names=["strain", "sample"], index_col="strain", squeeze=True
)
mgen_to_sample = library_to_sample.loc[mgen.sample]


focal_mgen_list = ['SS01009.m']
focal_mgen_consensus = mgen_consensus.mlift('sel', strain=focal_mgen_list)
focal_sample_list = mgen_to_sample.loc[focal_mgen_list].unique()
focal_strain_list = idxwhere((inferred_comm.data.sel(sample=focal_mgen_list) > rabund_threshold).any("sample").to_series())
focal_scg_list = idxwhere(scg_to_sample.loc[idxwhere(scg_horizontal_coverage.to_series() > horizontal_coverage_threshold)].isin(focal_sample_list))
focal_scg = scg.mlift('sel', strain=focal_scg_list)
focal_geno = inferred_geno.mlift('sel', strain=focal_strain_list)


# comm_entropy2 = inferred_comm2.entropy("sample")
# geno_entropy2 = inferred_geno2.entropy("strain")
# focal_strain_list2 = idxwhere((inferred_comm2.data.sel(sample=focal_mgen_list) > rabund_threshold).any("sample").to_series())
# focal_geno2 = inferred_geno2.mlift('sel', strain=focal_strain_list2)

focal_mgen_list, focal_sample_list, focal_strain_list, len(focal_scg_list)

In [ ]:
len(library_to_sample.unique())

In [ ]:
inferred_comm.sel(strain=focal_strain_list, sample=focal_mgen_list).to_series()

In [ ]:
from scipy.spatial.distance import pdist, squareform

def masked_hamming_distance(x, y):
    mask = ~(np.isnan(x) | np.isnan(y))
    diff = np.abs(x - y)
    return diff[mask].sum() / np.ones_like(diff)[mask].sum()

def genotype_mhd_pdist(genotypes):
    return pd.DataFrame(squareform(pdist(genotypes.values, metric=masked_hamming_distance)), index=genotypes.strain, columns=genotypes.strain)

genotype_mhd_pdist(focal_scg.discretized())

In [ ]:
from sklearn.cluster import AgglomerativeClustering
clust = pd.Series(AgglomerativeClustering(n_clusters=None, distance_threshold=0.15, linkage='complete', affinity='precomputed').fit(genotype_mhd_pdist(focal_scg.discretized())).labels_, index=focal_scg.strain)
clust.value_counts()

In [ ]:
for x in inferred_comm.sel(strain=focal_strain_list, sample=focal_mgen_list[0]).to_series().sort_values(ascending=False).iteritems():
    print(x)

In [ ]:
together = sf.data.Genotypes.concat(dict(
    scg=focal_scg,
    mgen=focal_mgen_consensus,
#     dmgen=focal_mgen_consensus.discretized(),
#     strain0=focal_geno,
#     dstrain0=focal_geno.discretized(),
#     strain1=focal_geno2,
    strain=focal_geno,
), dim='strain')

meta=pd.DataFrame(dict(
    geno_type=together.strain.to_series().str.split('_', 1).apply(lambda x: x[0]),
    original_label=together.strain.to_series().str.split('_', 1).apply(lambda x: x[1]),
), index=together.strain).fillna(-1)
meta['clust'] = clust.rename(lambda s: 'scg_' + s)
meta['clust'] = meta.clust.fillna(-1)
meta['zorder'] = meta.geno_type.map(dict(scg=0, strain=1, mgen=2))
meta['show'] = (meta.geno_type == 'scg')  # | (meta.geno_type == 'mgen')


# meta['markersize'] = meta.geno_type.map({'scg': 40, 'mgen': 100, 'dstrain0': 100})

strain_markersize = pd.concat([
    _comm.sel(strain=_strain_list, sample=focal_mgen_list[0]).to_series().rename(lambda i: _label + '_' + str(i)) * 400
    for _label, _comm, _strain_list in [
#         ('strain1', inferred_comm, focal_strain_list),
        ('strain', inferred_comm, focal_strain_list)
#         ('strain0', inferred_comm, focal_strain_list),
#         ('strain1', inferred_comm2, focal_strain_list2)
    ]
])

meta.loc[strain_markersize.index, 'markersize'] = strain_markersize
meta.loc[meta.geno_type == 'scg', 'markersize'] = scg_horizontal_coverage.to_series().rename(lambda s: 'scg_' + s) * 400
meta.loc[meta.geno_type == 'mgen', 'markersize'] = 100

#meta['markersize']

colorby = 'clust'
color_palette = {-1.0: 'black', 0.0: 'lightblue', 1.0: 'violet', 2.0: 'lightgreen', 3.0: 'peachpuff', 4.0: 'orange'}

markerby = 'geno_type'
marker_palette = dict(scg='o', strain='^', mgen='s')

fig, ax = plt.subplots(figsize=(5, 5))

ax, ordin, *_ = lib.plot.ordination_plot(
    genotype_mhd_pdist(together.discretized()),
    ordin=lib.plot.pca_ordination,
    meta=meta,
    showby='show',
#     colorby='geno_type',
    colorby=colorby,
    color_palette=color_palette,
#     color_palette=dict(scg='black', strain0='lightblue', dstrain0='blue', strain1='violet', dstrain1='purple', mgen='green'),
    markerby=markerby,
#     marker_palette={-1.0: 'o', 0.0: 'v', 1.0: '>', 2.0: '<', 3.0: 's'},
    marker_palette=marker_palette,
    zorderby='zorder',
#     zorderby_palette=dict(scg=0, strain0=1, dstrain0=2, strain1=1, dstrain1=3, mgen=3),
    markersizeby='markersize',
#     markersize_palette=markersize_palette,
    ordin_kws=dict(),
#     fill_legend=False,
    scatter_kws=dict(lw=0.2, alpha=1.0),
    ax=ax,
    fill_legend=False,
)

ax.set_xlabel('NMDS Axis 1')
ax.set_ylabel('NMDS Axis 2')

for i, (_strain, _) in enumerate(inferred_comm.sel(strain=focal_strain_list, sample=focal_mgen_list[0]).to_series().sort_values(ascending=False).iteritems(), start=1):
    for prefix in ['strain']:
        _key = f'{prefix}_{_strain}'
        ordin_data = ordin.loc[_key]
        x, y = tuple(ordin.loc[_key][['PC1', 'PC2']].to_list())
        offset = meta.loc[_key, 'markersize']**(1/3) / 100
        ax.scatter(x, y, marker='v', facecolor='none', edgecolor='black', s=ordin_data['markersize'], zorder=3, lw=2)
        ax.annotate(i, xy=(x, y + offset), ha='center', va='center')



for i, clust_letter in zip(sorted(meta.clust.unique()), ['__nolegend__', 'A', 'B', 'C', 'D']):
    if i == -1:
        continue
    plt.scatter([], [], marker=marker_palette['scg'], color=color_palette[i], lw=0.2, edgecolor='grey', label=f'Cluster {clust_letter}')
    
_key = f'mgen_{focal_mgen_list[0]}'
ordin_data = ordin.loc[_key]
x, y = tuple(ordin.loc[_key][['PC1', 'PC2']].to_list())
ax.scatter(x, y, marker='X', facecolor='red', edgecolor='grey', alpha=0.9, zorder=4, s=ordin_data['markersize'], label='Metagenotype')

        
for gen_type_label, _gen_type in [
#     ('Metagenotype', 'mgen'),
#     ('StrainFacts', 'dstrain0'),
#     ('StrainFacts', 'strain0'),
    ('StrainFacts', 'strain'),
#     ('StrainFacts fuzzy', 'strain0'),
]:
    plt.scatter([], [], marker=marker_palette[_gen_type], edgecolor='black', facecolor='None', label=gen_type_label)

ax.legend(bbox_to_anchor=(1., 1.0), ncol=1)
ax.get_legend().remove()

fig.savefig('fig/scg_ordination_104345_full.pdf', dpi=400, pad_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(2, 2))

for i, clust_letter in zip(sorted(meta.clust.unique()), ['__nolegend__', 'A', 'B', 'C', 'D']):
    if i == -1:
        continue
    plt.scatter([], [], marker=marker_palette['scg'], color=color_palette[i], lw=0.2, edgecolor='grey', label=f'Cluster {clust_letter}')
    
_key = f'mgen_{focal_mgen_list[0]}'
ordin_data = ordin.loc[_key]
x, y = tuple(ordin.loc[_key][['PC1', 'PC2']].to_list())
ax.scatter([], [], marker='X', facecolor='red', edgecolor='grey', alpha=0.9, zorder=4, s=ordin_data['markersize'], label='Metagenotype')

        
for gen_type_label, _gen_type in [
#     ('Metagenotype', 'mgen'),
#     ('StrainFacts', 'dstrain0'),
#     ('StrainFacts', 'strain0'),
    ('StrainFacts', 'strain'),
#     ('StrainFacts fuzzy', 'strain0'),
]:
    plt.scatter([], [], marker=marker_palette[_gen_type], edgecolor='black', facecolor='None', label=gen_type_label)

ax.legend()
ax.axis('off')

fig.savefig('fig/scg_ordination_104345_legend.pdf', dpi=400)

In [ ]:
scg_matching = pd.DataFrame(dict(
    strain=sf.evaluation.match_genotypes(
        focal_scg.discretized().to_world(),
        focal_geno.discretized().to_world(),
        cdist=lambda xx, yy: cdist(xx, yy, metric=masked_hamming_distance),
    )[0],
    strain_dist=sf.evaluation.match_genotypes(
        focal_scg.discretized().to_world(),
        focal_geno.discretized().to_world(),
        cdist=lambda xx, yy: cdist(xx, yy, metric=masked_hamming_distance),
    )[1],
    clust=clust,
)).rename_axis(index='scg')

(
    scg_matching
    .groupby(['clust', 'strain'])
    .strain_dist
    .agg(['count', 'median'])
    .apply(lambda x: f'{x["count"]} ({x["median"]:0.2})', axis=1)
    .unstack('clust', fill_value=0)
    .join(inferred_comm.sel(strain=focal_strain_list, sample=focal_mgen_list[0]).to_series())
    .sort_values('communities', ascending=False)
)

In [ ]:
scg_matching.groupby(['clust', 'strain']).agg(['count', 'mean'])#[d.scg_match==24]#

In [ ]:
inferred_comm.sel(strain=focal_strain_list, sample=focal_mgen_list[0]).to_series()

In [ ]:
together = sf.data.Genotypes.concat(dict(
    scg=focal_scg,
    mgen=focal_mgen_consensus,
#     dmgen=focal_mgen_consensus.discretized(),
#     strain0=focal_geno,
#     dstrain0=focal_geno.discretized(),
    strain1=focal_geno,
#     dstrain1=focal_geno2.discretized(),
), dim='strain')

sf.plot.plot_genotype(together, transpose=True, scalex=1e-3)

In [ ]:
together = sf.data.Genotypes.concat(dict(
    scg=focal_scg,
    mgen=focal_mgen_consensus,
#     dmgen=focal_mgen_consensus.discretized(),
#     strain0=focal_geno,
#     dstrain0=focal_geno.discretized(),
#     strain1=focal_geno2,
    strain=focal_geno,
), dim='strain')

meta=pd.DataFrame(dict(
    geno_type=together.strain.to_series().str.split('_', 1).apply(lambda x: x[0]),
    original_label=together.strain.to_series().str.split('_', 1).apply(lambda x: x[1]),
), index=together.strain).fillna(-1)
meta['clust'] = clust.rename(lambda s: 'scg_' + s)
meta['clust'] = meta.clust.fillna(-1)
meta['zorder'] = meta.geno_type.map(dict(scg=0, strain=1, mgen=2))
meta['show'] = (meta.geno_type == 'scg')  # | (meta.geno_type == 'mgen')


# meta['markersize'] = meta.geno_type.map({'scg': 40, 'mgen': 100, 'dstrain0': 100})

strain_markersize = pd.concat([
    _comm.sel(strain=_strain_list, sample=focal_mgen_list[0]).to_series().rename(lambda i: _label + '_' + str(i)) * 400
    for _label, _comm, _strain_list in [
#         ('strain1', inferred_comm, focal_strain_list),
        ('strain', inferred_comm, focal_strain_list)
#         ('strain0', inferred_comm, focal_strain_list),
#         ('strain1', inferred_comm2, focal_strain_list2)
    ]
])

meta.loc[strain_markersize.index, 'markersize'] = strain_markersize
meta.loc[meta.geno_type == 'scg', 'markersize'] = scg_horizontal_coverage.to_series().rename(lambda s: 'scg_' + s) * 400
meta.loc[meta.geno_type == 'mgen', 'markersize'] = 100

#meta['markersize']

colorby = 'clust'
color_palette = {-1.0: 'black', 0.0: 'lightblue', 1.0: 'violet', 2.0: 'lightgreen', 3.0: 'peachpuff', 4.0: 'orange'}

markerby = 'geno_type'
marker_palette = dict(scg='o', strain='^', mgen='s')

fig, ax = plt.subplots(figsize=(5, 5))

ax, ordin, *_ = lib.plot.ordination_plot(
    genotype_mhd_pdist(together.discretized()),
    ordin=lib.plot.pca_ordination,
    meta=meta,
    showby='show',
#     colorby='geno_type',
    colorby=colorby,
    color_palette=color_palette,
#     color_palette=dict(scg='black', strain0='lightblue', dstrain0='blue', strain1='violet', dstrain1='purple', mgen='green'),
    markerby=markerby,
#     marker_palette={-1.0: 'o', 0.0: 'v', 1.0: '>', 2.0: '<', 3.0: 's'},
    marker_palette=marker_palette,
    zorderby='zorder',
#     zorderby_palette=dict(scg=0, strain0=1, dstrain0=2, strain1=1, dstrain1=3, mgen=3),
    markersizeby='markersize',
#     markersize_palette=markersize_palette,
    ordin_kws=dict(),
#     fill_legend=False,
    scatter_kws=dict(lw=0.2, alpha=1.0),
    ax=ax,
    fill_legend=False,
)

ax.set_xlabel('NMDS Axis 1')
ax.set_ylabel('NMDS Axis 2')

# for i, (_strain, _) in enumerate(inferred_comm.sel(strain=focal_strain_list, sample=focal_mgen_list[0]).to_series().sort_values(ascending=False).iteritems()):
#     for prefix in ['strain']:
#         _key = f'{prefix}_{_strain}'
#         ordin_data = ordin.loc[_key]
#         x, y = tuple(ordin.loc[_key][['PC1', 'PC2']].to_list())
#         offset = meta.loc[_key, 'markersize']**(1/3) / 100
#         ax.scatter(x, y, marker='v', facecolor='none', edgecolor='black', s=ordin_data['markersize'], zorder=3, lw=2)
#         ax.annotate(i, xy=(x, y + offset), ha='center', va='center')



# for i, clust_letter in zip(sorted(meta.clust.unique()), ['__nolegend__', 'A', 'B', 'C', 'D']):
#     if i == -1:
#         continue
#     plt.scatter([], [], marker=marker_palette['scg'], color=color_palette[i], lw=0.2, edgecolor='grey', label=f'Cluster {clust_letter}')
    
# _key = f'mgen_{focal_mgen_list[0]}'
# ordin_data = ordin.loc[_key]
# x, y = tuple(ordin.loc[_key][['PC1', 'PC2']].to_list())
# ax.scatter(x, y, marker='X', facecolor='red', edgecolor='grey', alpha=0.9, zorder=4, s=ordin_data['markersize'], label='Metagenotype')

        
# for gen_type_label, _gen_type in [
# #     ('Metagenotype', 'mgen'),
# #     ('StrainFacts', 'dstrain0'),
# #     ('StrainFacts', 'strain0'),
#     ('StrainFacts', 'strain'),
# #     ('StrainFacts fuzzy', 'strain0'),
# ]:
#     plt.scatter([], [], marker=marker_palette[_gen_type], edgecolor='black', facecolor='None', label=gen_type_label)

# ax.legend(bbox_to_anchor=(1., 1.0), ncol=1)
ax.get_legend().remove()

fig.savefig('fig/scg_ordination_104345_only_scg.png', dpi=400, pad_inches='tight')

In [ ]:
together = sf.data.Genotypes.concat(dict(
    scg=focal_scg,
    mgen=focal_mgen_consensus,
#     dmgen=focal_mgen_consensus.discretized(),
#     strain0=focal_geno,
#     dstrain0=focal_geno.discretized(),
#     strain1=focal_geno2,
    strain=focal_geno,
), dim='strain')

meta=pd.DataFrame(dict(
    geno_type=together.strain.to_series().str.split('_', 1).apply(lambda x: x[0]),
    original_label=together.strain.to_series().str.split('_', 1).apply(lambda x: x[1]),
), index=together.strain).fillna(-1)
meta['clust'] = clust.rename(lambda s: 'scg_' + s)
meta['clust'] = meta.clust.fillna(-1)
meta['zorder'] = meta.geno_type.map(dict(scg=0, strain=1, mgen=2))
meta['show'] = (meta.geno_type == 'scg')  # | (meta.geno_type == 'mgen')


# meta['markersize'] = meta.geno_type.map({'scg': 40, 'mgen': 100, 'dstrain0': 100})

strain_markersize = pd.concat([
    _comm.sel(strain=_strain_list, sample=focal_mgen_list[0]).to_series().rename(lambda i: _label + '_' + str(i)) * 400
    for _label, _comm, _strain_list in [
#         ('strain1', inferred_comm, focal_strain_list),
        ('strain', inferred_comm, focal_strain_list)
#         ('strain0', inferred_comm, focal_strain_list),
#         ('strain1', inferred_comm2, focal_strain_list2)
    ]
])

meta.loc[strain_markersize.index, 'markersize'] = strain_markersize
meta.loc[meta.geno_type == 'scg', 'markersize'] = scg_horizontal_coverage.to_series().rename(lambda s: 'scg_' + s) * 400
meta.loc[meta.geno_type == 'mgen', 'markersize'] = 100

#meta['markersize']

colorby = 'clust'
color_palette = {-1.0: 'black', 0.0: 'lightblue', 1.0: 'violet', 2.0: 'lightgreen', 3.0: 'peachpuff', 4.0: 'orange'}

markerby = 'geno_type'
marker_palette = dict(scg='o', strain='^', mgen='s')

fig, ax = plt.subplots(figsize=(5, 5))

ax, ordin, *_ = lib.plot.ordination_plot(
    genotype_mhd_pdist(together.discretized()),
    ordin=lib.plot.pca_ordination,
    meta=meta,
    showby='show',
#     colorby='geno_type',
    colorby=colorby,
    color_palette=color_palette,
#     color_palette=dict(scg='black', strain0='lightblue', dstrain0='blue', strain1='violet', dstrain1='purple', mgen='green'),
    markerby=markerby,
#     marker_palette={-1.0: 'o', 0.0: 'v', 1.0: '>', 2.0: '<', 3.0: 's'},
    marker_palette=marker_palette,
    zorderby='zorder',
#     zorderby_palette=dict(scg=0, strain0=1, dstrain0=2, strain1=1, dstrain1=3, mgen=3),
    markersizeby='markersize',
#     markersize_palette=markersize_palette,
    ordin_kws=dict(),
#     fill_legend=False,
    scatter_kws=dict(lw=0.2, alpha=1.0),
    ax=ax,
    fill_legend=False,
)

ax.set_xlabel('NMDS Axis 1')
ax.set_ylabel('NMDS Axis 2')

# for i, (_strain, _) in enumerate(inferred_comm.sel(strain=focal_strain_list, sample=focal_mgen_list[0]).to_series().sort_values(ascending=False).iteritems()):
#     for prefix in ['strain']:
#         _key = f'{prefix}_{_strain}'
#         ordin_data = ordin.loc[_key]
#         x, y = tuple(ordin.loc[_key][['PC1', 'PC2']].to_list())
#         offset = meta.loc[_key, 'markersize']**(1/3) / 100
#         ax.scatter(x, y, marker='v', facecolor='none', edgecolor='black', s=ordin_data['markersize'], zorder=3, lw=2)
#         ax.annotate(i, xy=(x, y + offset), ha='center', va='center')



# for i, clust_letter in zip(sorted(meta.clust.unique()), ['__nolegend__', 'A', 'B', 'C', 'D']):
#     if i == -1:
#         continue
#     plt.scatter([], [], marker=marker_palette['scg'], color=color_palette[i], lw=0.2, edgecolor='grey', label=f'Cluster {clust_letter}')
    
_key = f'mgen_{focal_mgen_list[0]}'
ordin_data = ordin.loc[_key]
x, y = tuple(ordin.loc[_key][['PC1', 'PC2']].to_list())
ax.scatter(x, y, marker='X', facecolor='red', edgecolor='grey', alpha=0.9, zorder=4, s=ordin_data['markersize'], label='Metagenotype')

        
# for gen_type_label, _gen_type in [
# #     ('Metagenotype', 'mgen'),
# #     ('StrainFacts', 'dstrain0'),
# #     ('StrainFacts', 'strain0'),
#     ('StrainFacts', 'strain'),
# #     ('StrainFacts fuzzy', 'strain0'),
# ]:
#     plt.scatter([], [], marker=marker_palette[_gen_type], edgecolor='black', facecolor='None', label=gen_type_label)

# ax.legend(bbox_to_anchor=(1., 1.0), ncol=1)
ax.get_legend().remove()

fig.savefig('fig/scg_ordination_104345_scg_and_mgen.png', dpi=400, pad_inches='tight')